In [1]:
%pwd

'c:\\Users\\User\\OneDrive\\Desktop\\NIT\\Chatbot_langchain\\AdI-Psychotherapist-Chatbot\\research'

In [2]:
import os
os.chdir("..\\")

In [3]:
%pwd

'c:\\Users\\User\\OneDrive\\Desktop\\NIT\\Chatbot_langchain\\AdI-Psychotherapist-Chatbot'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files(data="Data/")


In [9]:
len(extracted_data)

561

In [10]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [11]:
texts_chunk = text_split(extracted_data)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 5374


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

In [13]:
embedding = download_embeddings()

C:\Users\User\AppData\Local\Temp\ipykernel_12160\3120106633.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\User\anaconda3\envs\adibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
vector = embedding.embed_query("Hello world")
len(vector)

384

In [60]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=".env")

True

In [66]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

print("Pinecone Key Loaded:", PINECONE_API_KEY is not None)

Pinecone Key Loaded: True


In [17]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [18]:
from pinecone import ServerlessSpec 

index_name = "adibot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [23]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [24]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [34]:
retrieved_docs = retriever.invoke("What is anxiety?")

In [35]:
retrieved_docs

[Document(id='d70ba308-d26b-4ddc-9867-e42dc67bac8b', metadata={'author': 'Andrés J. Consoli,Larry E. Beutler,Bruce Bongar', 'creationdate': '2017-01-29T13:58:19+00:00', 'creator': 'Adobe InDesign CC 2015 (Windows)', 'ebx_publisher': 'OUP Premium', 'moddate': '2017-01-29T15:40:13+00:00', 'page': 56.0, 'page_label': '34', 'producer': 'Adobe PDF Library 15.0; modified using iText® 5.1.0 ©2000-2011 1T3XT BVBA', 'source': 'Data\\Comprehensive Textbook.pdf', 'title': 'Comprehensive Textbook of Psychotherapy', 'total_pages': 561.0, 'trapped': '/False'}, page_content='anxiety completely turned around the psychoanalytic \nunderstanding of its role in symptom formation and \nits use in psychotherapy. Neurotic anxiety was now \nunderstood to be the source of unconscious efforts at \nrepression and defense, rather than the result of these \nprocesses. Anxiety resulted from and was a signal of \nconflicts between the hypothesized structures of the \nmind (id and ego, or id and superego) and was the

In [65]:
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling f5074b1221da:   0% ▕                  ▏ 488 KB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   0% ▕                  ▏ 771 KB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   0% ▕                  ▏ 931 KB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   0% ▕                  ▏ 939 KB/4.4 GB   

In [74]:
from langchain_community.chat_models import ChatOllama

chatModel = ChatOllama(model="mistral", temperature=0.7,max_tokens=400)


In [75]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [76]:
system_prompt = (
    "You are an Psychotherapist assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. And that you dont' have that much knowledge."
    "Use three sentences maximum and keep the "
    "answer concise."
    "Always remind the person asking the question to "
    "consult a professional for serious issues."
    "\n\n"
    "{context}"
)


In [77]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [78]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [79]:
response = rag_chain.invoke({"input": "What is anxiety?"})
answer = response["answer"]

import time, sys

for word in answer.split():
    sys.stdout.write(word + " ")
    sys.stdout.flush()
    time.sleep(0.15)  # adjust typing speed (seconds per word)

print()  # final newline


Anxiety is a psychological state characterized by feelings of worry, fear, and apprehension, often associated with physical symptoms like increased heart rate, sweating, and trembling. In psychoanalytic theory, it's understood as a result and signal of conflicts between the hypothetical structures of the mind (id, ego, or superego), triggering defense mechanisms. For treatment, exposure therapy and motivational interviewing are often used, as outlined in works such as Abramowitz et al. (2011) and Westra (2012). However, for serious issues, it's always recommended to consult a professional psychotherapist. 


In [82]:
response = rag_chain.invoke({"input": "I'm feeling stressed!!"})
print(response["answer"])

 It seems like you're struggling with managing your emotions and finding ways to cope effectively. In therapy, it's essential to identify triggers for anger and frustration. You mentioned a desire for confrontation. It might be helpful to discuss this desire further in sessions and explore healthier outlets for expressing these feelings.

Regarding the weight gain, it's important to prioritize self-care and maintain a balanced lifestyle that includes regular exercise and a nutritious diet. This can help improve overall wellbeing and reduce stress levels.

Lastly, remember that therapy is a process, and its benefits may not be immediately noticeable. It's crucial to practice self-compassion and understand that progress takes time. If you're feeling overwhelmed or have concerns about your mental health, consult a professional for further guidance.


In [81]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

 Acne is a common skin condition characterized by areas of skin having small red, tender bumps (papules), larger red tender bumps with pus at the center (pustules), or even whiteheads, blackheads, and cysts. It often appears on the face, shoulders, back, and chest. Acne is most common among teenagers, though it can affect adults as well. If severe, acne can cause scarring. It's important to consult a dermatologist for proper treatment and management of acne.


In [83]:
response = rag_chain.invoke({"input": "what is Statistics?"})
print(response["answer"])

 Statistics is a branch of mathematics that deals with the collection, analysis, interpretation, and presentation of data. In psychology, statistics are used to make sense of collected data by identifying patterns, relationships, and differences.

Traditional statistical analyses in psychology involve comparing mean scores of group participants, both within and between groups. This can include pre-to-posttreatment comparisons and comparisons between treatment groups.

Significant differences are typically determined based on a 5% chance level (p < .05). However, it's important to note that the choice of statistical test depends on various factors and may require expertise in statistics. Always consult with a professional for serious issues or complex analyses.
